In [2]:
import sys
sys.path.append(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9\timeseires')

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [4]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [5]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [6]:
model1 = create_rnn()
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 24, 8)             240       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 20)                580       
                                                                 
 flatten (Flatten)           (None, 20)                0         
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 841
Trainable params: 841
Non-trainable params: 0
_________________________________________________________________


In [7]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [8]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [10]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [11]:
import os
path_dataset =C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9\\AEP_hourly_2'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [12]:
time_steps=24
num_features=21

In [13]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 1.8698985576629639 sec


In [14]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/60
2651/2653 [============================>.] - ETA: 0s - loss: 0.0432 - mae: 0.0432 - mape: 287.8660
Epoch 1: val_loss improved from inf to 0.01441, saving model to C:\Users\musta\Desktop\ML lab\lab 9\E1-cp-0001-loss0.01.h5
2653/2653 [==============================] - 74s 26ms/step - loss: 0.0432 - mae: 0.0432 - mape: 287.6988 - val_loss: 0.0144 - val_mae: 0.0144 - val_mape: 7.4275
Epoch 2/60
2650/2653 [============================>.] - ETA: 0s - loss: 0.0140 - mae: 0.0140 - mape: 356.5611
Epoch 2: val_loss did not improve from 0.01441
2653/2653 [==============================] - 52s 20ms/step - loss: 0.0140 - mae: 0.0140 - mape: 356.2209 - val_loss: 0.0168 - val_mae: 0.0168 - val_mape: 8.6580
Epoch 3/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0123 - mae: 0.0123 - mape: 305.1355
Epoch 3: val_loss improved from 0.01441 to 0.01256, saving model to C:\Users\musta\Desktop\ML lab\lab 9\E1-cp-0003-loss0.01.h5
2653/2653 [==============================] - 52s 20

2652/2653 [============================>.] - ETA: 0s - loss: 0.0086 - mae: 0.0086 - mape: 53.4591
Epoch 24: val_loss did not improve from 0.00777
2653/2653 [==============================] - 47s 18ms/step - loss: 0.0086 - mae: 0.0086 - mape: 53.4481 - val_loss: 0.0097 - val_mae: 0.0097 - val_mape: 4.5056
Epoch 25/60
2650/2653 [============================>.] - ETA: 0s - loss: 0.0084 - mae: 0.0084 - mape: 31.7562
Epoch 25: val_loss did not improve from 0.00777
2653/2653 [==============================] - 55s 21ms/step - loss: 0.0084 - mae: 0.0084 - mape: 31.7286 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 3.9567
Epoch 26/60
2651/2653 [============================>.] - ETA: 0s - loss: 0.0084 - mae: 0.0084 - mape: 60.8586
Epoch 26: val_loss did not improve from 0.00777
2653/2653 [==============================] - 70s 26ms/step - loss: 0.0084 - mae: 0.0084 - mape: 60.8241 - val_loss: 0.0086 - val_mae: 0.0086 - val_mape: 3.7615
Epoch 27/60
2652/2653 [============================>.] - E

2653/2653 [==============================] - ETA: 0s - loss: 0.0077 - mae: 0.0077 - mape: 253.0774
Epoch 53: val_loss did not improve from 0.00706
2653/2653 [==============================] - 80s 30ms/step - loss: 0.0077 - mae: 0.0077 - mape: 253.0774 - val_loss: 0.0102 - val_mae: 0.0102 - val_mape: 4.1305
Epoch 54/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0077 - mae: 0.0077 - mape: 283.6000
Epoch 54: val_loss did not improve from 0.00706
2653/2653 [==============================] - 97s 37ms/step - loss: 0.0077 - mae: 0.0077 - mape: 283.6000 - val_loss: 0.0071 - val_mae: 0.0071 - val_mape: 3.0414
Epoch 55/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0076 - mae: 0.0076 - mape: 141.7097
Epoch 55: val_loss did not improve from 0.00706
2653/2653 [==============================] - 93s 35ms/step - loss: 0.0076 - mae: 0.0076 - mape: 141.7097 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 5.0870
Epoch 56/60
2652/2653 [============================>

In [15]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9\E1-cp-0056-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 8s 11ms/step
Mean Absolute Error (MAE): 109.06
Median Absolute Error (MedAE): 87.51
Mean Squared Error (MSE): 20597.92
Root Mean Squared Error (RMSE): 143.52
Mean Absolute Percentage Error (MAPE): 0.74 %
Median Absolute Percentage Error (MDAPE): 0.6 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [16]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9\E1-cp-0039-loss0.01.h5'
start_epoch= 40

In [17]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\musta\Desktop\ML lab\lab 9\E1-cp-0039-loss0.01.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [18]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0067 - mae: 0.0067 - mape: 86.5551
Epoch 1: val_loss improved from inf to 0.00681, saving model to C:\Users\musta\Desktop\ML lab\lab 9\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 91s 32ms/step - loss: 0.0067 - mae: 0.0067 - mape: 86.5551 - val_loss: 0.0068 - val_mae: 0.0068 - val_mape: 3.0075
Epoch 2/10
2652/2653 [============================>.] - ETA: 0s - loss: 0.0067 - mae: 0.0067 - mape: 63.2501
Epoch 2: val_loss improved from 0.00681 to 0.00675, saving model to C:\Users\musta\Desktop\ML lab\lab 9\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 71s 27ms/step - loss: 0.0067 - mae: 0.0067 - mape: 63.2369 - val_loss: 0.0067 - val_mae: 0.0067 - val_mape: 3.0087
Epoch 3/10
2651/2653 [============================>.] - ETA: 0s - loss: 0.0066 - mae: 0.0066 - mape: 120.6042
Epoch 3: val_loss improved from 0.00675 to 0.00670, saving model to C:\Users\musta\Desktop\ML lab\lab 

In [19]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 9\E2-cp-0009-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 3s 6ms/step
Mean Absolute Error (MAE): 103.86
Median Absolute Error (MedAE): 82.84
Mean Squared Error (MSE): 18813.43
Root Mean Squared Error (RMSE): 137.16
Mean Absolute Percentage Error (MAPE): 0.71 %
Median Absolute Percentage Error (MDAPE): 0.58 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
